In [3]:
import numpy as np
from ase.io import read

# Path template for xyz files
path_template = 'Z:\\electrocyclic\\open\\open_g2m05_{:04d}\\open_g2m05_{:04d}.xyz'
num_files = 1000

# Initialize lists to store data
positions_list = []
atomic_numbers_list = []

# Iterate over each xyz file
for i in range(num_files):
    filename = path_template.format(i,i)
    atoms = read(filename)  # Read the xyz file
    
    # Extract atom positions and atomic numbers
    positions = atoms.get_positions()
    atomic_numbers = atoms.get_atomic_numbers()
    
    # Append to lists
    positions_list.append(positions)
    atomic_numbers_list.append(atomic_numbers)

# Convert lists to numpy arrays
positions_array = np.array(positions_list)
atomic_numbers_array = np.array(atomic_numbers_list)

# Save to npy files
np.save('positions.npy', positions_array)
np.save('atomic_numbers.npy', atomic_numbers_array)


In [7]:
positions_array[0]

array([[-2.00000000e-07,  1.00000000e-08,  9.79567120e-01],
       [ 1.06394980e-01,  1.25143584e+00,  4.12216340e-01],
       [ 1.38621050e-01,  1.63906707e+00, -8.83222640e-01],
       [-1.38621070e-01, -1.63906705e+00, -8.83222670e-01],
       [-1.06395000e-01, -1.25143584e+00,  4.12216310e-01],
       [ 1.40000000e-07, -4.00000000e-08,  1.97459065e+00],
       [ 1.72522750e-01,  2.02231093e+00,  1.16788128e+00],
       [ 2.28441480e-01,  2.69485781e+00, -1.08695179e+00],
       [ 8.13444800e-02,  9.73788830e-01, -1.72292100e+00],
       [-8.13448200e-02, -9.73788780e-01, -1.72292103e+00],
       [-2.28441320e-01, -2.69485781e+00, -1.08695183e+00],
       [-1.72522470e-01, -2.02231098e+00,  1.16788124e+00]])

In [6]:
atomic_numbers_array[0]

array([7, 6, 6, 6, 6, 1, 1, 1, 1, 1, 1, 1])

In [4]:
import numpy as np
from ase import Atoms
from schnetpack.data import ASEAtomsData

positions = np.load('positions.npy')
atomic_numbers = np.load('atomic_numbers.npy')
fock_matrices = np.load('fockmatrices_open.npy')
property_list = [{'fock_matrix': fock_matrix} for fock_matrix in fock_matrices]

# Create list of Atoms objects
atoms_list = []
for pos, nums in zip(positions, atomic_numbers):
    atoms = Atoms(positions=pos, numbers=nums)
    atoms_list.append(atoms)

# Create ASEAtomsData database
ase_db_path = 'my_test_new_hope.db'
new_dataset = ASEAtomsData.create(
    ase_db_path,
    distance_unit='Ang',
    property_unit_dict={'fock_matrix': 'unknown'}
)

# Add systems to the database
new_dataset.add_systems(property_list, atoms_list)


In [5]:
import numpy as np
import ase
from ase.io import read 
from ase.db import connect
import json
import matplotlib.pylab as plt
import os
import torch
import schnetpack as spk
import schnetpack.transform as trn
from torch.optim import Adam
from schnetpack.representation import SchNet
from schnetpack.atomistic import Atomwise, PairwiseDistances
from schnetpack.nn import GaussianRBF, CosineCutoff
from schnetpack.model import NeuralNetworkPotential
import torchmetrics

In [6]:
print('Number of reference calculations:', len(new_dataset))
print('Available properties:')

for p in new_dataset.available_properties:
    print('-', p)
print()

example = new_dataset[0]
print('Properties of molecule with id 0:')

for k, v in example.items():
    print('-', k, ':', v.shape)

Number of reference calculations: 1000
Available properties:
- fock_matrix

Properties of molecule with id 0:
- _idx : torch.Size([1])
- fock_matrix : torch.Size([32, 32])
- _n_atoms : torch.Size([1])
- _atomic_numbers : torch.Size([12])
- _positions : torch.Size([12, 3])
- _cell : torch.Size([1, 3, 3])
- _pbc : torch.Size([3])
